# AMR-Wind simulation post-processing & analysis 
# -- Flow Field --

## Import necessary functions

In [1]:
# Add any possible locations of submodules here
amrwindfedirs = ['/projects/wind_uq/lcheung/amrwind-frontend',
                 '/ascldap/users/lcheung/wind_uq/amrwind-frontend/',
                 '/ccs/proj/cfd162/lcheung/amrwind-frontend/',
                '/home/jfrederi/projects/amrwind-frontend'
                '/home/jfrederi/projects/moa_python']
import sys, os, shutil, scipy, pandas
for x in amrwindfedirs: sys.path.insert(1, x)

import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib as mpl
import netCDF4 as ncdf
import pandas as pd
import os
%matplotlib notebook
from scipy.optimize import curve_fit

from moa_python.post_plane import Post_plane

### Set data folder(s)

In [2]:
base_folder = '/projects/ssc/jfrederi/amr-wind-runs/snl_precursors/medWS_lowTI'

## Define cases
case_folder = ['baseline_snl','ccwhelix_snl','sideside_snl','wakesteering_-20deg_snl']
legend_names = ['Baseline','Helix','Pulse','Wake steering']

suffix = "post_processing"
for case in case_folder:
    plane_folder = os.path.join(base_folder, case, suffix)
    available_planes = [(i) for i in os.listdir(plane_folder) if i.endswith(".nc") & ~i.startswith("abl")]
    print(f"Case '{case}' contains the following planes: \n{available_planes}")

Case 'baseline_snl' contains the following planes: 
['XZdomaincoarse_68125.nc', 'YZwake_68125.nc', 'XZ_68125.nc', 'XY_68125.nc', 'YZinflow_68125.nc', 'XYdomaincoarse_68125.nc', 'YZcoarse_68125.nc']
Case 'ccwhelix_snl' contains the following planes: 
['XZdomaincoarse_68125.nc', 'YZwake_68125.nc', 'XZ_68125.nc', 'XY_68125.nc', 'YZinflow_68125.nc', 'XYdomaincoarse_68125.nc', 'YZcoarse_68125.nc']
Case 'sideside_snl' contains the following planes: 
['YZwake_123125.nc', 'XZdomaincoarse_68125.nc', 'YZwake_118125.nc', 'YZwake_68125.nc', 'XZ_118125.nc', 'XY_68125.nc', 'YZinflow_68125.nc', 'XZdomaincoarse_83125.nc', 'YZcoarse_68125.nc']
Case 'wakesteering_-20deg_snl' contains the following planes: 
['YZwake_68125.nc', 'XY_68125.nc']


### Import planes of interest

In [ ]:
## Choose planes to load
hor_plane_file = 'XY_68125.nc'
ver_plane_file = 'YZwake_68125.nc'

## Settings
sampling_freq = 1
Ncases = len(case_folder)
hub_height = 150
D = 240
turbine_hor_location = [1200, 480, 150]

## Load planes
hor_plane = []
ver_plane = []
for n in range(Ncases):
    plane_folder = os.path.join(base_folder, case_folder[n], suffix)
    hor_plane.append(Post_plane(os.path.join(plane_folder, hor_plane_file), freq = sampling_freq))
    hor_plane[n].set_origin_simple(x=turbine_hor_location)
    hor_plane[n].scale_to_rot_diam(D)
    hor_plane[n].set_turbine_whereabouts(D = 1, turb_loc = [0, 0, 0])
    
    ver_plane.append(Post_plane(os.path.join(plane_folder, ver_plane_file), freq = sampling_freq))
    ver_plane[n].set_origin_simple([480, 0, 0], frame='absolute')
    ver_plane[n].scale_to_rot_diam(D)
    ver_plane[n].set_turbine_whereabouts(D = 1, turb_loc = [0, hub_height/D, 0])

Plane has 5 plane(s) in 2401 time steps from 27250.0 to 28450.0
Plane offsets: [ 30. 150. 270. 390. 510.]
Plane has 22 plane(s) in 2401 time steps from 27250.0 to 28450.0
Plane offsets: [1224. 1224. 1260. 1320. 1380. 1440. 1500. 1560. 1680. 1800. 1920. 2040.
 2160. 2400. 2640. 2880. 3120. 3360. 3600. 3840. 4080. 4320.]
Plane has 5 plane(s) in 2401 time steps from 27250.0 to 28450.0
Plane offsets: [ 30. 150. 270. 390. 510.]
Plane has 22 plane(s) in 2401 time steps from 27250.0 to 28450.0
Plane offsets: [1224. 1224. 1260. 1320. 1380. 1440. 1500. 1560. 1680. 1800. 1920. 2040.
 2160. 2400. 2640. 2880. 3120. 3360. 3600. 3840. 4080. 4320.]
Plane has 5 plane(s) in 801 time steps from 27250.0 to 27650.0
Plane offsets: [ 30. 150. 270. 390. 510.]
Plane has 22 plane(s) in 801 time steps from 27250.0 to 27650.0
Plane offsets: [1224. 1224. 1260. 1320. 1380. 1440. 1500. 1560. 1680. 1800. 1920. 2040.
 2160. 2400. 2640. 2880. 3120. 3360. 3600. 3840. 4080. 4320.]


### Plot horizontal plane (average and instantaneous)

In [ ]:
## Set parameters
hub_height = 0
St = 0.3
U = 9.05
helix_period = D/(St*U)
timespan = 8*helix_period

## Set figure
fig, axes = plt.subplots(Ncases,1,figsize = (9.5,4*Ncases), tight_layout=True)
if not isinstance(axes, (list, np.ndarray)): axes = [axes]
plt.set_cmap('coolwarm')

## Plot subfigures for all cases
for n in range(Ncases):
    tend = hor_plane[n].time[-1]
    hor_plane[n].plot_mean_plane(hub_height, ax = axes[n], timespan = [tend-timespan,tend])
    axes[n].set_title(case_folder[n])
    
    # Remove colorbar from subplots
    im = [obj for obj in axes[n].get_children() if isinstance(obj, mpl.collections.Collection)][0]
    if hasattr(im,'colorbar') and im.colorbar is not None:
        im.colorbar.remove()
fig.colorbar(im,ax = axes[n],location='bottom')

In [ ]:
## Select time instance to plot
time_instance = 28200

## Set figure
fig, axes = plt.subplots(Ncases,1,figsize = (9.5,4*Ncases), tight_layout=True)
if not isinstance(axes, (list, np.ndarray)): axes = [axes]
plt.set_cmap('coolwarm')

## Plot subfigures for all cases
for n in range(Ncases):
    tend = hor_plane[n].time[-1]
    hor_plane[n].plot_plane(hub_height, ax = axes[n], time = time_instance)
    axes[n].set_title(case_folder[n])
    
    # Remove colorbar from subplots
    im = [obj for obj in axes[n].get_children() if isinstance(obj, mpl.collections.Collection)][0]
    if hasattr(im,'colorbar') and im.colorbar is not None:
        im.colorbar.remove()
fig.colorbar(im,ax = axes[n],location='bottom')

### Plot average vertical planes

In [ ]:
## Select planes to plot
downstream_distances = [1, 3, 5, 7]
planes_to_plot = np.array(downstream_distances)
Nplanes = len(planes_to_plot)

# Set up figure
fig, axes = plt.subplots(Nplanes, Ncases, figsize = (8, 4*Ncases), layout='compressed')
if not isinstance(axes[0],(list,np.ndarray)): axes = [axes]
    
## Plot planes for all cases
for k in range(Nplanes):
    for n in range(Ncases):
        tend = ver_plane[n].time[-1]
        ver_plane[n].plot_mean_plane(planes_to_plot[k], timespan = [tend-timespan, tend], \
                                     ax = axes[k][n], verbose=False)
        ver_plane[n].plot_turbine(ax=axes[k][n], plane='yz')
        axes[k][n].set_title(f'{legend_names[n]}, {downstream_distances[k]}D')
        im = [obj for obj in axes[k][n].get_children() if isinstance(obj, mpl.collections.Collection)][0]
        if hasattr(im,'colorbar') and im.colorbar is not None:
            im.colorbar.remove()
        if k > 0: axes[k][n].set_ylabel('')

# fig.subplots_adjust(bottom=0.9)
# cbar_ax = fig.add_axes([0.9, 0.2, 0.03, 0.6])
# fig.colorbar(im, cax=cbar_ax, shrink=0.5)
# fig.savefig('ws_downstream.png',dpi=400)


## Velocity in wake

In [ ]:
## Set location
fig_wake, ax_wake = plt.subplots(1, 1, figsize=(10,5), tight_layout=True)
legend_string = []

## Get velocities in wake and plot
wake_hor_vel = []
wake_cs_vel = np.empty((Ncases, len(ver_plane[0].z)))
for n in range(Ncases):
    tend = hor_plane[n].time[-1]
    color = 'C'+str(n)
    hor_plane_w = hor_plane[n].plot_vel_in_wake(timespan = [tend-timespan,tend], ax = ax_wake, color = color, verbose = False)
    ver_plane_w = ver_plane[n].plot_vel_in_wake(z = ver_plane[n].z, timespan = [tend-timespan,tend], axis='z', ax = ax_wake, \
                                linestyle = 'o', color=color, verbose = False)
    wake_hor_vel = np.append(wake_hor_vel, np.mean(hor_plane_w))
    legend_string += [f'{legend_names[n]}, streamwise @hub height',f'{legend_names[n]}, cross section']

ax_wake.legend(legend_string,loc='lower right')
ax_wake.set_title('Velocity in the wake');


## Power of virtual turbines

In [ ]:
## Get T1 data

vars_and_labels = {'Time':('Time','s'),
            'GenPwr':('Gen. Power','W')}
vars = list(vars_and_labels.keys())
turbines = ['T0']
windcases = [9.05,]

openfastData = {}
counter = 0
for caseiter, case in enumerate(case_folder):
    openfastData[case] = {}
    for mps in windcases:
        openfastData[case][str(mps)] = {}
        dfs = []
        for turbiter, turbine in enumerate(turbines):
            for variter , var in enumerate(vars):
                file = base_folder + '/' + case_folder[caseiter] + '/OpenFAST/' + turbine + '_' + var + '.dat'
                print('Reading from: ', file)
                df = pd.read_csv(file,sep=' ',skiprows=(1,))

                df.drop_duplicates(subset='Time', inplace=True)
                if variter == 0:
                    dfs = df
                if variter > 0:
                    dfs = pd.merge(dfs, df, on="Time")
            openfastData[case][str(mps)][str(turbine)] = dfs

In [ ]:
pwrT1 = []
for n, case in enumerate(case_folder):
    idx0 = np.where(openfastData[case][str(mps)][str(turbine)]['Time'] 
                    > openfastData[case][str(mps)][str(turbine)]['Time'].iloc[-1]-timespan)[0][0]
    pwrT1.append(np.mean(openfastData[case][str(mps)][str(turbine)]['GenPwr'][idx0:]))

In [ ]:
fig, ax = plt.subplots(Ncases-1, 1, figsize=(9.5, 10), tight_layout=True)
levels = [0.9, 1, 1.05, 1.1, 1.15]
angle = [0, 0, -20]

for n in range(Ncases):
    tend = hor_plane[n].time[-1]
    hor_plane[n].ravfield = hor_plane[n].get_virtual_turbine_power(output = 'power', power_T1 = pwrT1[n]*1e3, 
                                                        cp = 0.44, timespan = [tend-timespan,tend])
    if n > 0:
        hor_plane[n].plot_virtual_turbine_field(hor_plane[n].ravfield, baseline_field=hor_plane[0].ravfield, filter_order = 3, angle=angle[n-1],
                                                vmin = 0.85, vmax = 1.2, turb_loc=hor_plane[n].turb_loc, ax = ax[n-1], contour_levels = levels)
        ax[n-1].set_title(f"{legend_names[n]}: Estimated power gain for downstream turbine locations")
        
## Add colorbar to bottom
im = [obj for obj in ax[-1].get_children() if isinstance(obj, mpl.collections.Collection)][0]
fig.colorbar(im,ax = ax[-1],location='bottom', shrink=.8)

fig.savefig('est_power_2T_WF.png', dpi = 720)

In [ ]:
fig, ax = plt.subplots(Ncases-1, 1, figsize=(9.5, 10))
levels = [1, 1.02, 1.04, 1.06]

for n in range(Ncases):
    ver_plane[n].ravfield = ver_plane[n].get_virtual_turbine_power(output = 'power', power_T1 = pwrT1[n]*1e3, axis = 'z', cp = 0.44, timespan = timespan)
    if n > 0:
        ver_plane[n].plot_virtual_turbine_field(ver_plane[n].ravfield.T, x = ver_plane[n].z, y = ver_plane[n].x, baseline_field=ver_plane[0].ravfield.T, 
                                                ax = ax[n-1], contour_levels = levels)
        ax[n-1].set_title(f"Total power gain for downstream turbine location, {legend_names[n]}")

In [ ]:
x = ver_plane[0].x
y = 0
z = ver_plane[0].z

X = np.array(np.meshgrid(x, y, z)).reshape((3, np.size(x)*np.size(y)*np.size(z)))

field = []
for xi, yi, zi in zip(X[0][:], X[1][:], X[2][:]):
    field.append(ver_plane[0].mean_vel_in_wake(D, [xi, yi, 0], zi, axis='z', verbose = False))


In [ ]:
x = ver_plane[0].x
y = 0
z = ver_plane[0].z
baseline_field = ver_plane[0].ravfield
fig, ax = plt.subplots()
for n in range(Ncases-1):
    ax.plot(x, ver_plane[n+1].ravfield[13,:]/baseline_field[13,:])
    ax.grid(True)
    ax.legend(legend_names[1:])


In [ ]:
pwrT1